In [1]:
#CONDA ENV base_conda (python 3.9.7)
#Import packages
#---------------------------------------
import sys
import os
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm

#Import your modules
#---------------------------------------
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm

# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
s_fig = '/cndd3/dburrows/FIGS/'

%load_ext autoreload
sys.version

'3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) \n[GCC 9.4.0]'

In [31]:
os.chdir('/cndd3/dburrows/DATA/public_datasets/splice/smartseq_mouse_p53-59_MOp_pachter21/')
prac = pd.read_csv('intron.counts.csv.gz')
! ls
#awk '{$a=$5-$4;print $a}' GSM4800806_PFC_sc_Rep1_PacBio_processed.gff | head -50

cluster.annotation.csv	exon.counts.csv.gz    QC.csv
cluster.membership.csv	intron.counts.csv.gz  sample_metadata.csv.gz


In [30]:
meta

,Unnamed: 0,ar_id,exp_component_vendor_name,batch,batch_vendor_name,tube,tube_internal_name,tube_contents_nm,tube_contents_nm_from_vendor,tube_avg_size_bp,...,Injection_type,NewBatch,Vendor,MultiPlex,percent_reads_aligned_to_rrna_rmsk,percent_reads_aligned_to_trna_rmsk,percent_reads_aligned_to_ncrna,percent_reads_aligned_to_introns,percent_reads_aligned_to_intergenic,percent_reads_aligned_unique
0,LS-15395_S41_E1-50,543183476,LS-15395-41,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.162980,9.855384,3.264771,84.993663
1,LS-15395_S42_E1-50,543183474,LS-15395-42,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,16.281620,6.763949,2.856108,87.252036
2,LS-15395_S43_E1-50,543183472,LS-15395-43,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,14.448787,13.283785,4.473082,84.439773
3,LS-15395_S44_E1-50,543183470,LS-15395-44,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,16.687114,7.724415,3.083142,87.400895
4,LS-15395_S46_E1-50,543183466,LS-15395-46,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.844740,8.100115,3.478772,85.846808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,SM-GE92M_S092_E1-50,708586022,SM-GE92M-S092,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.692675,0.054178,4.007977,12.591947,27.800393,NaN
6296,SM-GE92M_S093_E1-50,708586372,SM-GE92M-S093,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.409496,0.051422,3.830221,9.094367,29.950267,NaN
6297,SM-GE92M_S094_E1-50,708586078,SM-GE92M-S094,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.610336,0.086938,3.419039,9.457431,29.709904,NaN
6298,SM-GE92M_S095_E1-50,708586242,SM-GE92M-S095,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.403770,0.040953,2.808983,6.679797,24.221576,NaN


In [20]:
#meta =  pd.read_csv('sample_metadata.csv.gz')
np.unique(np.asarray([i.split('-')[0] for i in meta['cell_name']]))


array(['Ctgf', 'Nr5a1', 'Pvalb', 'Rbp4', 'Scnn1a', 'Slc17a6', 'Slc17a7',
       'Slc32a1', 'Snap25'], dtype='<U7')

In [13]:
for i in meta.columns: 
    if 'cell' in i: print(i)


patched_cell_container
cell_name
cell_id
cell_prep_type


In [28]:
prac['Unnamed: 0'].values[-100:]

array(['Zmynd15', 'Zmynd19', 'Zmynd8', 'Znf157', 'Znf41-ps', 'Znf512b',
       'Znf660', 'Znfx1', 'Znhit1', 'Znhit2', 'Znhit3', 'Znhit6', 'Znrd1',
       'Znrd1as', 'Znrf1', 'Znrf2', 'Znrf3', 'Znrf4', 'Zp1', 'Zp2', 'Zp3',
       'Zp3r', 'Zp4-ps', 'Zpbp', 'Zpbp2', 'Zpld1', 'Zpr1', 'Zranb1',
       'Zranb2', 'Zranb3', 'Zrsr1', 'Zrsr2', 'Zscan10', 'Zscan12',
       'Zscan18', 'Zscan2', 'Zscan20', 'Zscan21', 'Zscan22', 'Zscan25',
       'Zscan26', 'Zscan29', 'Zscan30', 'Zscan4-ps1', 'Zscan4-ps2',
       'Zscan4-ps3', 'Zscan4a', 'Zscan4b', 'Zscan4c', 'Zscan4d',
       'Zscan4e', 'Zscan4f', 'Zscan5b', 'Zswim1', 'Zswim2', 'Zswim3',
       'Zswim4', 'Zswim5', 'Zswim6', 'Zswim7', 'Zswim8', 'Zufsp', 'Zw10',
       'Zwilch', 'Zwint', 'Zxda', 'Zxdb', 'Zxdc', 'Zyg11a', 'Zyg11b',
       'Zyx', 'Zzef1', 'Zzz3', 'a', 'l7Rn6', 'n-R5s100', 'n-R5s103',
       'n-R5s104', 'n-R5s105', 'n-R5s108', 'n-R5s110', 'n-R5s111',
       'n-R5s113', 'n-R5s117', 'n-R5s121', 'n-R5s122', 'n-R5s123',
       'n-R5s124', '

In [29]:
prac

,Unnamed: 0,LS-15395_S41_E1-50,LS-15395_S42_E1-50,LS-15395_S43_E1-50,LS-15395_S44_E1-50,LS-15395_S46_E1-50,LS-15395_S47_E1-50,LS-15395_S48_E1-50,LS-15395_S49_E1-50,LS-15395_S50_E1-50,...,SM-GE92M_S087_E1-50,SM-GE92M_S088_E1-50,SM-GE92M_S089_E1-50,SM-GE92M_S090_E1-50,SM-GE92M_S091_E1-50,SM-GE92M_S092_E1-50,SM-GE92M_S093_E1-50,SM-GE92M_S094_E1-50,SM-GE92M_S095_E1-50,SM-GE92M_S096_E1-50
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0610006L08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610007P14Rik,76,177,109,214,193,239,123,84,134,...,69,29,1,67,42,63,79,77,20,66
3,0610009B22Rik,188,224,67,59,174,413,82,127,177,...,90,26,22,57,48,66,63,92,87,0
4,0610009E02Rik,18,0,0,0,2,0,0,0,2,...,0,0,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45764,n-R5s143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45765,n-R5s144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45766,n-R5s146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
git add **
git commit -m 'pac'

In [32]:
prac

,Unnamed: 0,LS-15395_S41_E1-50,LS-15395_S42_E1-50,LS-15395_S43_E1-50,LS-15395_S44_E1-50,LS-15395_S46_E1-50,LS-15395_S47_E1-50,LS-15395_S48_E1-50,LS-15395_S49_E1-50,LS-15395_S50_E1-50,...,SM-GE92M_S087_E1-50,SM-GE92M_S088_E1-50,SM-GE92M_S089_E1-50,SM-GE92M_S090_E1-50,SM-GE92M_S091_E1-50,SM-GE92M_S092_E1-50,SM-GE92M_S093_E1-50,SM-GE92M_S094_E1-50,SM-GE92M_S095_E1-50,SM-GE92M_S096_E1-50
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,0610006L08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610007P14Rik,0,0,0,0,18,0,0,0,12,...,0,0,38,0,1,0,0,0,0,0
3,0610009B22Rik,0,0,0,1,0,0,0,0,0,...,1,3,0,6,2,9,3,1,1,0
4,0610009E02Rik,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45764,n-R5s143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45765,n-R5s144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45766,n-R5s146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
